Based on the papers:

@article{wood2021trading,
  title={Trading with the Momentum Transformer: An Intelligent and Interpretable Architecture},
  author={Wood, Kieran and Giegerich, Sven and Roberts, Stephen and Zohren, Stefan},
  journal={arXiv preprint arXiv:2112.08534},
  year={2021}
}

@article {Wood111,
	author = {Wood, Kieran and Roberts, Stephen and Zohren, Stefan},
	title = {Slow Momentum with Fast Reversion: A Trading Strategy Using Deep Learning and Changepoint Detection},
	volume = {4},
	number = {1},
	pages = {111--129},
	year = {2022},
	doi = {10.3905/jfds.2021.1.081},
	publisher = {Institutional Investor Journals Umbrella},
	issn = {2640-3943},
	URL = {https://jfds.pm-research.com/content/4/1/111},
	eprint = {https://jfds.pm-research.com/content/4/1/111.full.pdf},
	journal = {The Journal of Financial Data Science}
}

In [ ]:
!pip install empyrical-reloaded

In [14]:
from empyrical import (sharpe_ratio, max_drawdown, downside_risk, annual_return, annual_volatility,)
from typing import Dict, List, Optional, Tuple, Union
import pandas as pd
import numpy as np
import os

import warnings
warnings.filterwarnings('ignore')

In [15]:
def calc_returns(srs: pd.Series, day_offset: int = 1):
    returns = srs / srs.shift(day_offset) - 1
    return returns

In [16]:
def calc_daily_vol(daily_returns):
    return (
        daily_returns.ewm(span = 60, min_periods = 60).std().fillna(method="bfill")
    )

In [17]:
def calc_vol_scaled_returns(daily_returns, daily_vol=pd.Series(None)):
    if not len(daily_vol):
        daily_vol = calc_daily_vol(daily_returns)
    annualized_vol = daily_vol * np.sqrt(252)
    return daily_returns / annualized_vol.shift(1) #Had multiplication by target vol but don't care about that

In [18]:
class MACDStrat:
    def __init__(self, trend_combinations: List[Tuple[float, float]] = None):
        if trend_combinations is None:
            self.trend_combinations = [(8, 24), (16, 48), (32, 96)]
        else:
            self.trend_combinations = trend_combinations 
    
    @staticmethod
    def calc_signal(prices: pd.Series, short_timescale: int, long_timescale: int):

        def calc_halflife(timescale):
            return np.log(0.5) / np.log(1 - 1/timescale)
        
        macd = (
            prices.ewm(halflife= calc_halflife(short_timescale)).mean() - prices.ewm(halflife = calc_halflife(long_timescale)).mean()
        )

        q = macd / prices.rolling(63).std().fillna(method="bfill") #Standardize MACD with volatility 
        return q / q.rolling(252).std().fillna(method="bfill")


In [19]:
def read_changepoint_file(file_path: str, lookback_window_length: int):
    return (
        pd.read_csv(file_path, index_col=0, parse_dates=True)
        .fillna(method="ffill")
        .dropna() 
        .assign(
            cp_location_norm=lambda row: (row["t"] - row["cp_location"])/ lookback_window_length
        ) 
    )

In [20]:
def prepare_cpd_features(folder_path: str, lookback_window_length: int):
    return pd.concat(
        [
            read_changepoint_file(
                os.path.join(folder_path, f), lookback_window_length
            ).assign(ticker=os.path.splitext(f)[0])
            for f in os.listdir(folder_path)
        ]
    )

In [21]:
def deep_momentum_features(df_asset:pd.DataFrame):
    df_asset["srs"] = df_asset["close"]
    ewm = df_asset["srs"].ewm(halflife=252)
    means = ewm.mean()
    stds = ewm.std()
    df_asset["srs"] = np.minimum(df_asset["srs"], means + 5 * stds)
    df_asset["srs"] = np.maximum(df_asset["srs"], means - 5 * stds)

    df_asset["daily_returns"] = calc_returns(df_asset["srs"])
    df_asset["daily_vol"] = calc_daily_vol(df_asset["daily_returns"])

    df_asset["target_returns"] = calc_vol_scaled_returns(
        df_asset["daily_returns"], df_asset["daily_vol"]
    ).shift(-1)

    def calc_normalized_returns(day_offset):
        return (
            calc_returns(df_asset["srs"], day_offset) / df_asset["daily_vol"] / np.sqrt(day_offset)
        )

    df_asset["norm_daily_return"] = calc_normalized_returns(1)
    df_asset["norm_monthly_return"] = calc_normalized_returns(21)
    df_asset["norm_quarterly_return"] = calc_normalized_returns(63)
    df_asset["norm_biannual_return"] = calc_normalized_returns(126)
    df_asset["norm_annual_return"] = calc_normalized_returns(252)

    trend_combinations = [(8, 24), (16, 48), (32, 96)]
    for short_window, long_window in trend_combinations:
        df_asset[f"macd_{short_window}_{long_window}"] = MACDStrat.calc_signal(
            df_asset["srs"], short_window, long_window
        )

    # date features
    if len(df_asset):
        df_asset["day_of_week"] = df_asset.index.dayofweek
        df_asset["day_of_month"] = df_asset.index.day
        df_asset["week_of_year"] = df_asset.index.isocalendar().week
        df_asset["month_of_year"] = df_asset.index.month
        df_asset["year"] = df_asset.index.year
        df_asset["date"] = df_asset.index 
    else:
        df_asset["day_of_week"] = []
        df_asset["day_of_month"] = []
        df_asset["week_of_year"] = []
        df_asset["month_of_year"] = []
        df_asset["year"] = []
        df_asset["date"] = []
    
    return df_asset.dropna()

In [22]:
def include_changepoint_features(features: pd.DataFrame, cpd_folder_name: str, lookback_window_length: int):
    features = features.merge(
        prepare_cpd_features(cpd_folder_name, lookback_window_length)[
            ["ticker", "cp_location_norm", "cp_score"]
        ]
        .rename(
            columns={
                "cp_location_norm": f"cp_rl_{lookback_window_length}",
                "cp_score": f"cp_score_{lookback_window_length}"
            }
        )
        .reset_index(),
        on =["date", "ticker"]
    )

    features.index = features["date"]

    return features

In [ ]:
#Use the wrds data with date and price as a dataframe for input to deep_momentum_features()
#Then we pass this features df to include_changepoint_features() along with the cpd folder and window length to find this file and add to the features


#Then save features to a csv to import to backtest (This will need to be done for each company)


In [27]:
!pip install wrds

  Using cached wrds-3.2.0-py3-none-any.whl (13 kB)
  Using cached SQLAlchemy-2.0.36-cp310-cp310-win_amd64.whl (2.1 MB)
  Using cached psycopg2_binary-2.9.10-cp310-cp310-win_amd64.whl (1.2 MB)
  Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Using cached greenlet-3.1.1-cp310-cp310-win_amd64.whl (298 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import wrds

conn = wrds.Connection()

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [29]:
ticker = 'TWX'

query = f"""
SELECT DISTINCT
    d.date,
    n.ticker,
    d.prc / d.cfacpr as close
FROM
    crsp.dsf as d
JOIN 
    crsp.dsenames as n on d.permno = n.permno
WHERE
    n.ticker = '{ticker}'
    and date BETWEEN '2016-01-01' and '2023-12-31'
ORDER BY
    date
"""

df = conn.raw_sql(query)


In [30]:
df

,date,ticker,close
0,2016-01-04,TWX,64.92
1,2016-01-05,TWX,65.52
2,2016-01-06,TWX,68.62
3,2016-01-07,TWX,70.20
4,2016-01-08,TWX,71.17
...,...,...,...
612,2018-06-08,TWX,95.34
613,2018-06-11,TWX,96.17
614,2018-06-12,TWX,96.22
615,2018-06-13,TWX,97.95


In [224]:
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

features = deep_momentum_features(df.copy())
features = features.reset_index(drop=True)

In [225]:
features = include_changepoint_features(features, "Data/Changepoints/", 21)

In [226]:
features

,ticker,close,srs,daily_returns,daily_vol,target_returns,norm_daily_return,norm_monthly_return,norm_quarterly_return,norm_biannual_return,...,macd_16_48,macd_32_96,day_of_week,day_of_month,week_of_year,month_of_year,year,date,cp_rl_21,cp_score_21
date,,,,,,,,,,,,,,,,,,,,,
2017-02-02,WMT,66.70000,66.70000,0.007096,0.009402,-0.020090,0.754768,-0.662541,-0.502736,-0.810595,...,-1.647531,-1.080200,3,2,5,2,2017,2017-02-02,0.524545,0.592896
2017-02-03,WMT,66.50000,66.50000,-0.002999,0.009254,-0.010237,-0.324038,-0.874170,-0.578324,-0.893124,...,-1.628110,-1.093428,4,3,5,2,2017,2017-02-03,0.571681,0.622772
2017-02-06,WMT,66.40000,66.40000,-0.001504,0.009101,0.051079,-0.165233,-0.973521,-0.642175,-0.976762,...,-1.609017,-1.105373,0,6,6,2,2017,2017-02-06,0.632653,0.622263
2017-02-07,WMT,66.89000,66.89000,0.007380,0.009079,0.095429,0.812795,-0.482390,-0.455464,-0.862952,...,-1.577263,-1.111640,1,7,6,2,2017,2017-02-07,0.667275,0.663724
2017-02-08,WMT,67.81000,67.81000,0.013754,0.009305,0.126791,1.478103,-0.307179,-0.382246,-0.745974,...,-1.521098,-1.106757,2,8,6,2,2017,2017-02-08,0.714617,0.610603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-21,WMT,154.80000,154.80000,0.007091,0.013242,0.056852,0.535510,-0.112075,-0.442458,-0.028562,...,-0.771321,0.779996,3,21,51,12,2023,2023-12-21,0.333020,0.670246
2023-12-22,WMT,156.64999,156.64999,0.011951,0.013214,-0.007303,0.904411,0.211405,-0.375370,0.069571,...,-0.739761,0.767631,4,22,51,12,2023,2023-12-22,0.380051,0.648526
2023-12-26,WMT,156.41000,156.41000,-0.001532,0.012998,0.045548,-0.117865,0.037652,-0.363258,0.085179,...,-0.713329,0.752617,1,26,52,12,2023,2023-12-26,0.428099,0.614016


In [227]:
features.to_csv("Data/Finished_Datasets/WMT.csv")